In [13]:
# 数值运算
import math
import numpy as np

#读取/写入数据
import pandas as pd
import os
import csv

# 进度条
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# 学习曲线绘制
from torch.utils.tensorboard import SummaryWriter

In [14]:
def same_seed(seed): 
    '''修复随机数的可重复性'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [15]:
def train_valid_split(data_set, valid_ratio, seed):
    '''划分训练集和验证集'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

In [16]:
def predict(test_loader, model, device):
    # 预测函数
    model.eval()  #切换预测模式
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

### 数据集

In [12]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
            
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

### 模型

In [19]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # 定义网络结构 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

### 特征选择

In [23]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''选择有用的特征'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # 选择有用的特征
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

### 训练函数

In [25]:
def trainer(train_loader, valid_loader, model, config, device):

    # 定义损失函数
    criterion = nn.MSELoss(reduction='mean') 

    # 定义优化算法
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 

    # 绘制
    writer = SummaryWriter() 

    if not os.path.isdir('./models'):
        #创建保存模型的目录
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        # 切换训练模式
        model.train() 
        loss_record = []

        # 可视化
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # 梯度归零
            x, y = x.to(device), y.to(device)   # 使用cpu/gpu运行
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # 反向传播
            optimizer.step()                    # 更新参数
            step += 1
            loss_record.append(loss.detach().item())
            
            #在tqdm进度条上显示当前的epoch数和损失
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        # 切换验证模式
        model.eval() 
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 1024,      # 随机种子
    'select_all': True,   # 是否全部特征
    'valid_ratio': 0.2,   # 验证集 大小
    'n_epochs': 3000,     # 轮数          
    'batch_size': 1024, 
    'learning_rate': 1e-5,              
    'early_stop': 400,    # 如果模型在这么多连续的时间内没有改进，停止训练。     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}
device

'cpu'

In [36]:
# 随机种子
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./data/ml2022spring-hw1/covid.train.csv').values, pd.read_csv('./data/ml2022spring-hw1/covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# 训练集大小
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# 特征选择
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# 输出特征数量
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader将Pytorch数据集批量加载
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2160, 118) 
valid_data size: (539, 118) 
test_data size: (1078, 117)
number of features: 117


In [37]:
# 把你的模型和数据放在同一个计算设备上
model = My_Model(input_dim=x_train.shape[1]).to(device) 
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|██████████| 3/3 [00:00<00:00, 65.94it/s, loss=80.9]


Epoch [1/3000]: Train loss: 73.4526, Valid loss: 57.0994
Saving model with loss 57.099...


Epoch [2/3000]: 100%|██████████| 3/3 [00:00<00:00, 133.38it/s, loss=67.2]


Epoch [2/3000]: Train loss: 60.0700, Valid loss: 60.6242


Epoch [3/3000]: 100%|██████████| 3/3 [00:00<00:00, 75.06it/s, loss=57.3]


...

Epoch [2998/3000]: Train loss: 2.2684, Valid loss: 1.9580


Epoch [2999/3000]: 100%|██████████| 3/3 [00:00<00:00, 34.28it/s, loss=2.25]


Epoch [2999/3000]: Train loss: 2.0943, Valid loss: 2.6888


Epoch [3000/3000]: 100%|██████████| 3/3 [00:00<00:00, 134.05it/s, loss=2.18]


Epoch [3000/3000]: Train loss: 2.4895, Valid loss: 2.8601


In [38]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 92554), started 0:04:00 ago. (Use '!kill 92554' to kill it.)

In [40]:
def save_pred(preds, file):
    '''将预测保存到指定文件'''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv') 

100%|██████████| 2/2 [00:00<00:00, 158.38it/s]
